In [39]:
#meta 10/26/2020 File Reorg
#Organize Yelp reviews, from a pandas dataframe to a fs 2 folders for positive and negative reviews

#reqs
# need yelp_review_polarity_csv.tgz
# src: https://www.kaggle.com/ashishsalunkhe/yelp-review-polarity

#note: for fast.ai NLP Ch.10
#note: improve for efficiency

In [3]:
import pandas as pd
import numpy as np
import os


## 0. Load data
source: Kaggle  
dataset: Yelp_Review_Polarity  
https://www.kaggle.com/ashishsalunkhe/yelp-review-polarity

#### Descripiton
https://www.kaggle.com/ilhamfp31/yelp-review-dataset

The Yelp reviews polarity dataset is constructed by considering stars 1 and 2 negative, and 3 and 4 positive. For each polarity 280,000 training samples and 19,000 testing samples are take randomly. In total there are 560,000 trainig samples and 38,000 testing samples. Negative polarity is class 1, and positive class 2.

The files train.csv and test.csv contain all the training samples as comma-sparated values. There are 2 columns in them, corresponding to class index (1 and 2) and review text. The review texts are escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".

In [4]:
df = pd.read_csv("train.csv", names = ['rating', 'review'])
print(df.shape)
df.head()

(560000, 2)


,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [5]:
df.dtypes

rating     int64
review    object
dtype: object

### 0.1 Quick EDA 
Group by positive and negative

In [6]:
df.rating.value_counts()

2    280000
1    280000
Name: rating, dtype: int64

In [7]:
df_grp = df.groupby("rating")
df_grp.first()

,review
rating,
1,"Unfortunately, the frustration of being Dr. Go..."
2,Been going to Dr. Goldberg for over 10 years. ...


In [8]:
df_grp.describe()

review                                                                
         count  unique                                                top freq
rating                                                                        
1       280000  280000  Stay away from this dry cleaners in Chandler. ...    1
2       280000  280000  Really enjoyed my visit. Thank you betsie Boop...    1

In [9]:
df_grp.groups

{1: Int64Index([     0,      2,      3,      5,      6,      7,     10,     15,
                 16,     17,
             ...
             559982, 559983, 559985, 559988, 559990, 559992, 559993, 559994,
             559997, 559998],
            dtype='int64', length=280000),
 2: Int64Index([     1,      4,      8,      9,     11,     12,     13,     14,
                 19,     25,
             ...
             559980, 559981, 559984, 559986, 559987, 559989, 559991, 559995,
             559996, 559999],
            dtype='int64', length=280000)}

In [10]:
for name, group in df_grp:
    print(name, group)
    

1         rating                                             review
0            1  Unfortunately, the frustration of being Dr. Go...
2            1  I don't know what Dr. Goldberg was like before...
3            1  I'm writing this review to give you a heads up...
5            1  Wing sauce is like water. Pretty much a lot of...
6            1  Owning a driving range inside the city limits ...
...        ...                                                ...
559992       1  Long line, inefficient staff. Maybe my expecta...
559993       1  First: Please note that many people are review...
559994       1  I'm shocked everyone time I go to a Bouchon Ba...
559997       1  Phone calls always go to voicemail and message...
559998       1  Looks like all of the good reviews have gone t...

[280000 rows x 2 columns]
2         rating                                             review
1            2  Been going to Dr. Goldberg for over 10 years. ...
4            2  All the food is great here. B

## 1. Text Preprocessing for fast.ai NLP
Process date into folders

In [38]:
#for each group
for name, group in df_grp:
    #setup a folder
    folder = str(name)

    #df for each group
    #print(group.__class__)
    
    #write each record to a file
    for index, row in group.iterrows():
        #set file name
        file_name = folder + "/" + str(index) +".txt"
        #print(file_name, row["review"])
        
        #save each review to a separate file
        with open(file_name, 'w') as f:
            f.write(row["review"])
    

## Xtra

In [31]:
#$xtra - write each row of dataframe to a file - not efficient way but does the job
#vars
i = 0

#for each group
for name, group in df_grp:
    #setup a folder
    folder = str(name)

    #df for each group
    #print(group.__class__)
    
    #write each record to a file
    for index, row in group.iterrows():
        print(row.__class__)
        file_name = folder + "/" + str(index) +".txt"
        print(file_name, row["review"])
        with open(file_name, 'w') as f:
            f.write(row["review"])
        i+= 1
        
        #for quick testing
        if i>3: 
            break
    

<class 'pandas.core.series.Series'>
1/0.txt Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.
<class 'pandas.core.series.Series'>
1/2.txt I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in 